In [2]:
import pandas as pd
import ctypes
from ctypes import c_char_p, c_void_p, c_double
import json


In [3]:
def insToDf(path):
    data_nr_ins_4 = pd.read_csv(path, sep=' ', index_col=0, header=None)
    df_ins = data_nr_ins_4.copy()
    df_ins.index = df_ins.index.rename("ID")
    df_ins = df_ins.rename(columns = {1: "n", 2: "M", 3: "B"})
    df_ins.columns = [str(x) for x in df_ins.columns]
    df_ins["weights"] = df_ins.apply(lambda x: list(x[df_ins.columns[3::2]]), axis=1)
    df_ins["values"] = df_ins.apply(lambda x: list(x[df_ins.columns[4::2]]), axis=1)
    df_ins = df_ins.drop(df_ins.columns[3:-2], axis=1)
    df_ins.index = abs(df_ins.index)
    return df_ins


In [4]:
# Do the combinatorics
clib = ctypes.cdll.LoadLibrary('./knapsack.so')
clib.solve.argtypes = [c_char_p, c_char_p]
clib.solve.restype = c_double

In [5]:
def resToDf(path):
    data_nr_res_4 = pd.read_csv(path, sep=' ', index_col=0, header=None)
    df_res = data_nr_res_4.copy()

    df_res.index = df_res.index.rename("ID")
    df_res["solution"] = df_res.apply(lambda x: list(x[df_res.columns[2:]]), axis=1)
    df_res = df_res.drop(df_res.columns[2:-1], axis=1)
    df_res = df_res.rename(columns = {1: 'possible', 2: "value", 3:"solution"})
    df_res.index = abs(df_res.index)
    return df_res

In [6]:
def solToDf(path):
    data_nr_sol_4 = pd.read_csv(path, sep=' ', index_col=0, header=None)
    df_sol = data_nr_sol_4.copy()
    df_sol.index = df_sol.index.rename("ID")
    df_sol = df_sol.drop(df_sol.columns[-1], axis=1)
    df_sol["solution"] = df_sol.apply(lambda x: list(x[df_sol.columns[2:]]), axis=1)
    df_sol = df_sol.drop(df_sol.columns[2:-1], axis=1)
    df_sol = df_sol.rename(columns = {1: 'n', 2: "optimum"})
    # data_nr_4.index = data_nr_4.index.rename("ID")
    return df_sol


In [7]:
def check(df_ins, df_sol, df_res, path):
    possible = df_ins["B"] <= df_sol["optimum"]
    check = df_res['possible'] == possible
    ok = check.sum() == df_ins.shape[0]
    if (not ok):
        idx = df_ins[~check].index.values
        ERRORS[f"{path}:{idx}"] = (~check).sum()
        print(f"{path}:{idx} - {(~check).sum()}")

# mal = ~check
# df_res[mal]
# display(df_ins[484:485])
# display(df_res[484:485])
# display(df_sol[484:485])
# df_ins.iloc[485]
# display(df_res)
# display(df_sol)

In [ ]:
NUM = 10
NUMBERS = [4, 10, 15, 20, 22, 25, 27, 30, 32, 35, 37, 40]

ERRORS = {}
TIME_SPENT = {}

for n in NUMBERS:
    PATH_INS = f"./data/{'nr'}/{'NR'}{n}_{'inst'}.dat"
    PATH_SOL = f"./data/{'nr'}/{'NK'}{n}_{'sol'}.dat"
    PATH_RES = f"./data/{'nr'}/res/{'N'}{n}.csv"

    df_ins = insToDf(PATH_INS)
#     print(f"df_ins shape: {df_ins.shape}")

    time = clib.solve(PATH_INS.encode(), PATH_RES.encode())
    print(f"N={n}, time: {time} seconds")
    TIME_SPENT[NUM] = {"time": time}

    df_res = resToDf(PATH_RES)
#     print(f"df_res shape: {df_res.shape}")

    df_sol = solToDf(PATH_SOL)
#     print(f"df_sol shape: {df_sol.shape}")

    errors = check(df_ins, df_sol, df_res, PATH_RES)
#     print(ERRORS)

#     print(TIME_SPENT)
with open("./time_spent.json", "w") as f:
    f.write(json.dumps(TIME_SPENT))
with open("./errors.json", "w") as f:
    f.write(json.dumps(ERRORS))

N=4, time: 0.002436647 seconds
N=10, time: 0.008560687 seconds
N=15, time: 0.13911387 seconds
N=20, time: 3.220593536 seconds
N=22, time: 10.276774363 seconds
N=25, time: 86.253023671 seconds
N=27, time: 340.290631575 seconds


In [ ]:
ERRORS

In [ ]:
pd.read_json("./time_spent.json", orient="index")